In [5]:
import numpy as np
import pandas as pd
#get the data
dataset = pd.read_csv(r"C:\Users\gradadm09\Desktop\Natalie\Wayfair-Babson_HackathonData 2019.csv")
dataset.shape

(992977, 18)

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992977 entries, 0 to 992976
Data columns (total 18 columns):
VisitDate               992977 non-null object
UniqueVisitID           992977 non-null int64
VisitorGroup            992977 non-null object
PlatformUsed            992977 non-null object
VisitSource             992977 non-null object
BrowserName             992977 non-null object
OSName                  992977 non-null object
State                   408261 non-null object
Gender                  639596 non-null object
IncomeRange             383272 non-null object
ViewedProductInVisit    992977 non-null int64
ViewedSaleInVisit       992977 non-null int64
TotalPageViews          992977 non-null int64
PlacedSearch            992977 non-null int64
SecondsOnSite           992977 non-null int64
ClickedBanner           992977 non-null int64
AddedToBasket           992977 non-null int64
Purchased               992977 non-null int64
dtypes: int64(9), object(9)
memory usage: 136.4+ MB


In [7]:
data_deleted = dataset.drop(['IncomeRange','State','UniqueVisitID','VisitDate'],axis = 1)

In [8]:
data_deleted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992977 entries, 0 to 992976
Data columns (total 14 columns):
VisitorGroup            992977 non-null object
PlatformUsed            992977 non-null object
VisitSource             992977 non-null object
BrowserName             992977 non-null object
OSName                  992977 non-null object
Gender                  639596 non-null object
ViewedProductInVisit    992977 non-null int64
ViewedSaleInVisit       992977 non-null int64
TotalPageViews          992977 non-null int64
PlacedSearch            992977 non-null int64
SecondsOnSite           992977 non-null int64
ClickedBanner           992977 non-null int64
AddedToBasket           992977 non-null int64
Purchased               992977 non-null int64
dtypes: int64(8), object(6)
memory usage: 106.1+ MB


In [11]:
data_fillnan = data_deleted.fillna(method = 'ffill') # fill in the NaN using the row before the row with null value
data_fillasunknown = data_deleted.fillna("Unknown") # fill in the NaN using "Unknown"
data_fillall = data_deleted.replace("Unknown",np.nan).fillna(method = 'ffill') # replace "Unknown" using NaN, and fill in the NaN
data_deletenan = data_deleted.dropna(axis = 0,how = 'any') # drop all rows with null value
data_deleteall = data_deleted.replace("Unknown",np.nan).dropna(axis = 0,how = 'any') # drop all rows with null value or "Unknown" value
data_deleteage = data_deleted.drop(['Gender'],axis = 1) # drop Gender

In [17]:
data_fillall.head()
saved_data = data_fillall.to_csv()

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encode = LabelEncoder()
labels = ['VisitorGroup','PlatformUsed','VisitSource','BrowserName','OSName','Gender']

In [7]:
train = data_fillall # We only have Male or Female for Gender!
train.info()
new_labels = []
for label in labels:
    new_label = label+"_Code"
    new_labels.append(new_label)
    train[new_label] = label_encode.fit_transform(train[label]) # turn labels to digit labels(0,1) or (1,2,3,4)...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992977 entries, 0 to 992976
Data columns (total 14 columns):
VisitorGroup            992977 non-null object
PlatformUsed            992977 non-null object
VisitSource             992977 non-null object
BrowserName             992977 non-null object
OSName                  992977 non-null object
Gender                  992977 non-null object
ViewedProductInVisit    992977 non-null int64
ViewedSaleInVisit       992977 non-null int64
TotalPageViews          992977 non-null int64
PlacedSearch            992977 non-null int64
SecondsOnSite           992977 non-null int64
ClickedBanner           992977 non-null int64
AddedToBasket           992977 non-null int64
Purchased               992977 non-null int64
dtypes: int64(8), object(6)
memory usage: 106.1+ MB


In [8]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder() # same as the dummy variables
onehot_features = ['VisitorGroup_Code', 'PlatformUsed_Code', 'VisitSource_Code', 'BrowserName_Code', 'OSName_Code']

In [9]:
enc.fit(train[onehot_features])

c:\users\lindi\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [10]:
enc.categories_

[array([0., 1., 2.]),
 array([0., 1.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1., 2., 3., 4., 5., 6.])]

In [11]:
enc_res = enc.transform(train[onehot_features])

In [12]:
print(enc_res.toarray())

[[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]]


In [13]:
print(enc_res.toarray().shape)

(992977, 18)


In [14]:
final_features = ['ViewedProductInVisit','ViewedSaleInVisit','TotalPageViews','PlacedSearch','SecondsOnSite','ClickedBanner','AddedToBasket']
for i in new_labels:
    final_features.append(i)
final_features

['ViewedProductInVisit',
 'ViewedSaleInVisit',
 'TotalPageViews',
 'PlacedSearch',
 'SecondsOnSite',
 'ClickedBanner',
 'AddedToBasket',
 'VisitorGroup_Code',
 'PlatformUsed_Code',
 'VisitSource_Code',
 'BrowserName_Code',
 'OSName_Code',
 'Gender_Code']

In [15]:
onehot_final = list(set(final_features) - set(onehot_features))

In [16]:
onehot_final

['AddedToBasket',
 'PlacedSearch',
 'SecondsOnSite',
 'ViewedProductInVisit',
 'TotalPageViews',
 'ViewedSaleInVisit',
 'ClickedBanner',
 'Gender_Code']

In [17]:
all_x = train[final_features]
y = train['Purchased']
all_x.head()

,ViewedProductInVisit,ViewedSaleInVisit,TotalPageViews,PlacedSearch,SecondsOnSite,ClickedBanner,AddedToBasket,VisitorGroup_Code,PlatformUsed_Code,VisitSource_Code,BrowserName_Code,OSName_Code,Gender_Code
0,0,0,1,0,0,0,0,2,1,1,0,3,0
1,1,0,9,0,230,0,0,0,0,2,2,6,0
2,0,0,28,0,10412,0,0,0,1,2,0,5,0
3,1,1,8,1,430,0,0,2,0,0,2,6,0
4,1,1,9,0,1158,0,0,0,0,1,2,6,0


In [18]:
onehot_added = pd.DataFrame(enc_res.toarray())
all_x_2 = pd.concat([train[onehot_final],onehot_added],axis = 1)
all_x_2.head()

,AddedToBasket,PlacedSearch,SecondsOnSite,ViewedProductInVisit,TotalPageViews,ViewedSaleInVisit,ClickedBanner,Gender_Code,0,1,...,8,9,10,11,12,13,14,15,16,17
0,0,0,0,0,1,0,0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,0,230,1,9,0,0,0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0,10412,0,28,0,0,0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,1,430,1,8,1,0,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,0,1158,1,9,1,0,0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
all_x.isnull().sum()

ViewedProductInVisit    0
ViewedSaleInVisit       0
TotalPageViews          0
PlacedSearch            0
SecondsOnSite           0
ClickedBanner           0
AddedToBasket           0
VisitorGroup_Code       0
PlatformUsed_Code       0
VisitSource_Code        0
BrowserName_Code        0
OSName_Code             0
Gender_Code             0
dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
xTrain, xTest, yTrain, yTest = train_test_split(all_x, y, test_size = 0.2, random_state = 0)
xTrain.shape, xTest.shape

((794381, 13), (198596, 13))

In [22]:
yTrain.shape, yTest.shape

((794381,), (198596,))

In [23]:
yTrain.mean(), yTest.mean()

(0.02557085328073053, 0.02494007935708675)

In [24]:
x2Train, x2Test, y2Train, y2Test = train_test_split(all_x_2, y, test_size = 0.2, random_state = 0)

In [25]:
x2Train.mean(), x2Test.mean()

(AddedToBasket              0.106395
 PlacedSearch               0.366231
 SecondsOnSite           1436.790794
 ViewedProductInVisit       0.509295
 TotalPageViews            13.979803
 ViewedSaleInVisit          0.139652
 ClickedBanner              0.046439
 Gender_Code                0.187722
 0                          0.238780
 1                          0.210244
 2                          0.550976
 3                          0.510987
 4                          0.489013
 5                          0.180442
 6                          0.309195
 7                          0.510362
 8                          0.457330
 9                          0.055224
 10                         0.487446
 11                         0.212104
 12                         0.008459
 13                         0.000850
 14                         0.113830
 15                         0.000147
 16                         0.238695
 17                         0.425915
 dtype: float64, AddedToBasket        

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [27]:
logr = LogisticRegression()
dtree = DecisionTreeClassifier()
rf = RandomForestClassifier()
models = [logr,dtree,rf]

In [28]:
logr.fit(x2Train,y2Train)

c:\users\lindi\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
y_pred_test = logr.predict(x2Test)
y_pred_train = logr.predict(x2Train)

In [30]:
np.mean(y_pred_test == y2Test)

0.9748232592801467

In [31]:
np.mean(y_pred_train == y2Train)

0.9742302497164459

In [32]:
coefs = logr.coef_
coefs

array([[ 3.65820064e+00,  1.08670427e-01,  8.69431361e-06,
         3.36347986e-01,  4.09465993e-03, -1.68380870e-01,
        -1.79987932e-01,  2.81554986e-01, -1.32243530e+00,
        -5.51429634e-01, -3.10914021e-01, -1.25121483e+00,
        -9.33564117e-01, -3.50472033e-01, -8.62241857e-01,
        -9.72065061e-01, -5.88966307e-01, -7.31383005e-01,
        -8.64429639e-01, -9.27051099e-01, -1.54581494e-01,
        -1.31220722e-02, -1.70143917e-01, -6.20107016e-03,
        -4.19208065e-01, -4.94471233e-01]])

In [33]:
intercept = logr.intercept_
intercept

array([-2.18477895])

In [34]:
import statsmodels.api as sm
log_reg = sm.Logit(y2Train, x2Train)
clf = log_reg.fit()

Optimization terminated successfully.
         Current function value: 0.073912
         Iterations 10


In [35]:
clf.summary()

c:\users\lindi\appdata\local\programs\python\python37\lib\site-packages\statsmodels\base\model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
c:\users\lindi\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\users\lindi\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\users\lindi\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Purchased   No. Observations:               794381
Model:                          Logit   Df Residuals:                   794359
Method:                           MLE   Df Model:                           21
Date:                Fri, 01 Mar 2019   Pseudo R-squ.:                  0.3788
Time:                        11:58:45   Log-Likelihood:                -58714.
converged:                       True   LL-Null:                       -94525.
                                        LLR p-value:                     0.000
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
AddedToBasket            3.7059      0.026    143.444      0.000       3.655       3.757
PlacedSearch             0.1135      0.017      6.650      0.000       0.080       0.147
SecondsOnSite         8.826e-06   1.75e-06      5.047      0.000     5.4e-06    1.23e-05
ViewedProductInVisit     0.2467      0.036      6.940      0.000       0.177       0.316
TotalPageViews           0.0041      0.000     19.609      0.000       0.004       0.004
ViewedSaleInVisit       -0.1618      0.020     -8.106      0.000      -0.201      -0.123
ClickedBanner           -0.1947      0.034     -5.724      0.000      -0.261      -0.128
Gender_Code              0.2779      0.019     14.330      0.000       0.240       0.316
0                       -1.8518        nan        nan        nan         nan         nan
1                       -1.0440        nan        nan        nan         nan         nan
2                       -0.8094        nan        nan        nan         nan         nan
3                       -2.0145        nan        nan        nan         nan         nan
4                       -1.6908        nan        nan        nan         nan         nan
5                       -0.8620        nan        nan        nan         nan         nan
6                       -1.3675        nan        nan        nan         nan         nan
7                       -1.4758        nan        nan        nan         nan         nan
8                       -1.0765        nan        nan        nan         nan         nan
9                       -1.2597        nan        nan        nan         nan         nan
10                      -1.3691        nan        nan        nan         nan         nan
11                      -0.8003        nan        nan        nan         nan         nan
12                      -0.4269        nan        nan        nan         nan         nan
13                      -0.3807        nan        nan        nan         nan         nan
14                      -0.0504        nan        nan        nan         nan         nan
15                      -1.3802        nan        nan        nan         nan         nan
16                      -0.2931        nan        nan        nan         nan         nan
17                      -0.3737        nan        nan        nan         nan         nan
========================================================================================
"""